In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

About Breast Cancer Wisconsin (Diagnostic) Data Set Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image. n the 3-dimensional space is that described in: [K. P. Bennett and O. L. Mangasarian: "Robust Linear Programming Discrimination of Two Linearly Inseparable Sets", Optimization Methods and Software 1, 1992, 23-34].

This database is also available through the UW CS ftp server: ftp ftp.cs.wisc.edu cd math-prog/cpo-dataset/machine-learn/WDBC/

Also can be found on UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

Attribute Information:

1) ID number 2) Diagnosis (M = malignant, B = benign) 3-32)

Ten real-valued features are computed for each cell nucleus:

a) radius (mean of distances from center to points on the perimeter) b) texture (standard deviation of gray-scale values) c) perimeter d) area e) smoothness (local variation in radius lengths) f) compactness (perimeter^2 / area - 1.0) g) concavity (severity of concave portions of the contour) h) concave points (number of concave portions of the contour) i) symmetry j) fractal dimension ("coastline approximation" - 1)

The mean, standard error and "worst" or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features. For instance, field 3 is Mean Radius, field 13 is Radius SE, field 23 is Worst Radius.

All feature values are recoded with four significant digits.

Missing attribute values: none

Class distribution: 357 benign, 212 malignant

## 1.Import Libraries

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno 
from sklearn.model_selection import train_test_split

## 2.Explore Dataset

In [ ]:
data = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')
data.head()

In [ ]:
msno.matrix(data)

### Unnamed:32 column is full of NaN values, we will drop it

In [ ]:
#Eliminate NaN value column
data = data.iloc[:,:32]
data.head()

### Change "diagnosis" column values to numeric from string

In [ ]:
data.diagnosis.replace(['M','B'],[0,1],inplace = True)
data.head()

### Save 'id' column with assuming that we will make submission(because of there is no competition with the dataset, we will not make submission) and then drop it because there is need for it in training.

In [ ]:
id_c = data['id'] 
del data['id']
data.head()

## 3.Split Data into the Training set and Test set

In [ ]:
x_data = data.iloc[:,1:]
y_data = data.iloc[:,:1]
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.1, random_state = 0)

## 4.Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)


## Now data is ready. It's time to prepare classifier.

<a id="22"></a> <br>
## Implementing with keras library
Lets look at some parameters of keras library:
* units: output dimensions of node
* kernel_initializer: to initialize weights
* activation: activation function, we use relu
* input_dim: input dimension that is number of pixels in our images (4096 px)
* optimizer: we use adam optimizer
    * Adam is one of the most effective optimization algorithms for training neural networks.
    * Some advantages of Adam is that relatively low memory requirements and usually works well even with little tuning of hyperparameters
* loss: Cost function is same. By the way the name of the cost function is cross-entropy cost function that we use previous parts.
$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small \tag{6}$$
* metrics: it is accuracy.
* epochs: number of iteration
* Batch size defines number of samples that going to be propagated through the network.

In [ ]:
#Import Keras and its packages.
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

### We will create a function for our classifier

In [ ]:
classifier = Sequential() #Initialising the ANN
# Let's add input layer and first hidden layer
classifier.add(Dense(units =16, kernel_initializer = 'uniform', activation = 'relu', input_dim = x_train.shape[1]))
classifier.add(Dropout(0.01))
# Let's add 2 more hidden layer and output layer

#2 hidden layers
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.01))

classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.01))

# Output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

#Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#Fitting data
classifier.fit(x_train, y_train, epochs = 150, batch_size = 100)

### output_dim and units are the same. output_dim is 1 as we want only 1 output from the final layer.

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(x_test)
y_pred = (y_pred > 0.5)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print("Classifier accuracy : {}%".format(((cm[0][0] + cm[1][1])/57)*100))

In [ ]:
sns.heatmap(cm,annot=True)
plt.savefig('hm.png')

# Thank you for reading my notebook. I am open to suggestions if you have any. If you liked my notebook, please support and encourage me for new notebooks with upvoting 🙃🙂